In [6]:
import tkinter as tk
from tkinter import ttk
import numpy as np

class AffineTransformationApp:
    def __init__(self, root):
        self.root = root
        self.root.title('Affine Transformation')

        self.vertices = np.array([
            [300, 450, 1],
            [600, 450, 1],
            [450, 150, 1]
        ])

        self.canvas = tk.Canvas(root, width=1200, height=1000)
        self.canvas.pack()


        matrix_frame = tk.Frame(root)
        matrix_frame.pack()

        self.matrix_entries = []
        labels = ['a', 'b', 'e', 'c', 'd', 'f', ':', ':', ':']
        for i in range(3):
            row = []
            for j in range(3):
                label = tk.Label(matrix_frame, text=labels[i*3+j], width=2)
                label.grid(row=i, column=j*2)
                if i < 2:
                    entry = ttk.Entry(matrix_frame, width=5)
                    entry.insert(0, '1' if i == j else '0')
                    entry.grid(row=i, column=j*2+1)
                    row.append(entry)
                else:
                    label = tk.Label(matrix_frame, text=labels[i*3+j], width=5)
                    label.grid(row=i, column=j*2+1)
            if row:
                self.matrix_entries.append(row)
        self.transform_button = ttk.Button(root, text='Transform', command=self.apply_transformation)
        self.transform_button.pack()

        self.vertex_entries = []
        vertex_labels = ['Vertex 1 (x,y):', 'Vertex 2 (x,y):', 'Vertex 3 (x,y):']
        for i in range(3):
            label = tk.Label(matrix_frame, text=vertex_labels[i])
            label.grid(row=i, column=6)
            x_entry = ttk.Entry(matrix_frame, width=5)
            y_entry = ttk.Entry(matrix_frame, width=5)
            x_entry.insert(0, str(self.vertices[i][0]))
            y_entry.insert(0, str(self.vertices[i][1]))
            x_entry.grid(row=i, column=7)
            y_entry.grid(row=i, column=8)
            self.vertex_entries.append((x_entry, y_entry))
    
        self.update_triangle_button = ttk.Button(root, text='Update Triangle', command=self.update_triangle)
        self.update_triangle_button.pack()

        
        self.draw_triangle()
    def update_triangle(self):
        for i in range(3):
            x, y = self.vertex_entries[i]
            self.vertices[i][0] = float(x.get())
            self.vertices[i][1] = float(y.get())
        self.draw_triangle()

    def draw_triangle(self):
        self.canvas.delete('all')
    
        for i in range(3):
            next_i = (i + 1) % 3
            self.canvas.create_line(self.vertices[i][0], self.vertices[i][1],
                                    self.vertices[next_i][0], self.vertices[next_i][1])
    
        for i, (x_entry, y_entry) in enumerate(self.vertex_entries):
            x_entry.delete(0, tk.END)
            x_entry.insert(0, str(self.vertices[i][0]))
            y_entry.delete(0, tk.END)
            y_entry.insert(0, str(self.vertices[i][1]))

    def get_transformation_matrix(self):
        matrix = []
        for row in self.matrix_entries:
            matrix_row = []
            for entry in row:
                matrix_row.append(float(entry.get()))
            matrix.append(matrix_row)
        matrix.append([0, 0, 1])
        return np.array(matrix)

    def apply_transformation(self):
        transformation_matrix = self.get_transformation_matrix()
    
        centroid_x = np.mean(self.vertices[:, 0])
        centroid_y = np.mean(self.vertices[:, 1])
    
        to_origin = np.array([
            [1, 0, -centroid_x],
            [0, 1, -centroid_y],
            [0, 0, 1]
        ])
    
        back_to_place = np.array([
            [1, 0, centroid_x],
            [0, 1, centroid_y],
            [0, 0, 1]
        ])
        combined_transform = back_to_place @ transformation_matrix @ to_origin
    
        self.vertices = np.dot(self.vertices, combined_transform.T)
    
        self.draw_triangle()

root = tk.Tk()
app = AffineTransformationApp(root)

root.mainloop()
